# Lab Four: The Multi-Layer Perceptron
 

#### Everett Cienkus, Blake Miller, Colin Weil

### 1. Load, Split, and Balance

#### 1.1 Load Data

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load the data into memory and save it to a pandas data frame.
df = pd.read_csv('census_dataset/acs2017_census_tract_data.csv')

# Remove any observations that having missing data.
df = df.dropna()

# Encode any string data as integers for now. 
le = LabelEncoder()
s = le.fit_transform(df['State'])
df['State'] = s
c = le.fit_transform(df['County'])
df['County'] = c

X = df.drop(columns = ['ChildPoverty'])
y = df['ChildPoverty']

Discuss why we are going to keep/ not keep county variable.

#### 1.2 Split Dataset

In [2]:
# Divide your data into training and testing data using an 80% training 
# and 20% testing split. Use the cross validation modules that are part 
# of scikit-learn.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8)

unique_ytrain, counts_ytrain = np.unique(y_train, return_counts=True)
print(np.asarray((unique_ytrain, counts_ytrain)).T)

[[0.00e+00 3.91e+03]
 [1.00e-01 1.90e+01]
 [2.00e-01 2.20e+01]
 ...
 [9.78e+01 1.00e+00]
 [9.83e+01 1.00e+00]
 [1.00e+02 4.40e+01]]


#### 1.3 Balance (NOT DONE) Should we use state maybe? or just do the

In [29]:
print(len(X_train)/4)

X_train.head()

print(len(X_train[X_train.State < 10]))
# print(len(X_train[X_train.State > 9 & X_train.State < 20]))

# all factors are {1, 2, 103, 206, 353, 706, 36359, 72718}

14543.5
14424


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

We balance the training and not the testing set because it is good to balance for training because then we have information for all of our classes but when we are testing, we want to test on the distributuion that would actually be the case in order to get an accurate represenation of the results for testing.

### 2. Pre-proccesing

MEGNEMAR for comparing

### 3. Modeling

Feed forward/ back propogation in for-loop for adding layers

### 4. Adaptive momentum (AdaM) 